In [ ]:
# Install TensorFlow 2.x
!pip install -q tensorflow

# Imports
from pathlib import Path
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import (
    Conv1D, BatchNormalization, Activation, GlobalAveragePooling1D,
    Add, Dense, Input
)
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import os


In [ ]:
# Global Constants
WINDOW_SIZE = 60
NUM_FEATURES = 6
STRIDE = 15
ACCURACY_CUTOFF = 0.95
EPOCHS = 100

# Project Paths
PROJECT_DIR = Path("/content/drive/MyDrive/Colab_HAR_Project")
DATA_DIR = PROJECT_DIR / "data"
OUTPUT_DIR = PROJECT_DIR / "converted_test_model"
MODEL_SAVE_PATH = OUTPUT_DIR / "trained_tcn_model.h5"
SAVED_MODEL_DIR = str(OUTPUT_DIR / "fl_model_saved")

# Ensure output directory exists
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)


In [ ]:
def create_windows(df, window_size, stride):
    windows, labels = [], []
    if 'subject' not in df.columns:
        df['subject'] = 'default'
    for _, group_df in df.groupby(['subject', 'activity']):
        data = group_df[['x_accel', 'y_accel', 'z_accel', 'x_gyro', 'y_gyro', 'z_gyro']].values
        if len(data) < window_size:
            continue
        for start in range(0, len(data) - window_size + 1, stride):
            windows.append(data[start:start + window_size])
            labels.append(group_df["activity"].iloc[0])
    return np.array(windows, dtype=np.float32), np.array(labels)


def build_tcn_model(input_shape, num_classes):
    x = input_layer = Input(shape=input_shape)
    for rate in [1, 2, 4, 8, 16]:
        prev_x = x
        x = Conv1D(64, 7, dilation_rate=rate, padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        if prev_x.shape[-1] != x.shape[-1]:
            prev_x = Conv1D(64, 1, padding='same')(prev_x)
        x = Add()([prev_x, x])
    x = GlobalAveragePooling1D()(x)
    output_layer = Dense(num_classes, activation='softmax')(x)
    return Model(inputs=input_layer, outputs=output_layer)


In [ ]:
df1 = pd.read_csv(DATA_DIR / "resampled_normalized_phone_data.csv")
df2 = pd.read_csv(DATA_DIR / "combined_collected_data.csv")

combined_df = pd.concat([
    df1[df1['activity'].isin(['B', 'D', 'E'])],
    df2[df2['activity'].isin(['A', 'C'])]
], ignore_index=True)

X, y_raw = create_windows(combined_df, WINDOW_SIZE, STRIDE)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y_raw).astype(np.int32)
num_classes = len(label_encoder.classes_)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(-1, NUM_FEATURES)).reshape(X_train.shape)
X_val_scaled = scaler.transform(X_val.reshape(-1, NUM_FEATURES)).reshape(X_val.shape)

model = build_tcn_model((WINDOW_SIZE, NUM_FEATURES), num_classes)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

class EarlyStoppingByAccuracy(tf.keras.callbacks.Callback):
    def __init__(self, target_acc=ACCURACY_CUTOFF):
        super().__init__()
        self.target_acc = target_acc
    def on_epoch_end(self, epoch, logs=None):
        val_acc = logs.get("val_accuracy")
        if val_acc and val_acc >= self.target_acc:
            print(f"✅ Early stop: val_accuracy={val_acc:.4f} ≥ {self.target_acc}")
            self.model.stop_training = True

model.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=EPOCHS,
    batch_size=64,
    verbose=2,
    callbacks=[EarlyStoppingByAccuracy()]
)

model.save(MODEL_SAVE_PATH)
print(f"✅ Model saved at: {MODEL_SAVE_PATH}")


Epoch 1/100
✅ Early stop: val_accuracy=0.9558 ≥ 0.95
702/702 - 110s - 157ms/step - accuracy: 0.9377 - loss: 0.1696 - val_accuracy: 0.9558 - val_loss: 0.1085


✅ Model saved at: /content/drive/MyDrive/Colab_HAR_Project/converted_test_model/trained_tcn_model.h5


In [ ]:
class FederatedModelWrapper(tf.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.optimizer = tf.keras.optimizers.Adam()
        self.weight_specs = [
            tf.TensorSpec(shape=w.shape, dtype=w.dtype) for w in model.trainable_variables
        ]

    @tf.function(input_signature=[
        tf.TensorSpec([None, WINDOW_SIZE, NUM_FEATURES], tf.float32),
        tf.TensorSpec([None], tf.int32)
    ])
    def train(self, x, y):
        with tf.GradientTape() as tape:
            preds = self.model(x, training=True)
            loss = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(y, preds))
        grads = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.model.trainable_variables))
        acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(preds, axis=1, output_type=tf.int32), y), tf.float32))
        return {"loss": loss, "accuracy": acc}

    @tf.function(input_signature=[tf.TensorSpec([None, WINDOW_SIZE, NUM_FEATURES], tf.float32)])
    def infer(self, x):
        return {"predictions": self.model(x, training=False)}

    @tf.function(input_signature=[])
    def get_weights(self):
        return {f'w{i}': w for i, w in enumerate(self.model.trainable_variables)}

    @tf.function
    def set_weights(self, *weights):
        for var, val in zip(self.model.trainable_variables, weights):
            var.assign(val)
        return {"status": tf.constant("weights_set")}

    def get_signatures(self):
        return {
            "train": self.train.get_concrete_function(),
            "infer": self.infer.get_concrete_function(),
            "get_weights": self.get_weights.get_concrete_function(),
            "set_weights": self.set_weights.get_concrete_function(*self.weight_specs),
        }

# Load if needed
if model is None:
    model = tf.keras.models.load_model(MODEL_SAVE_PATH)

fl_model = FederatedModelWrapper(model)
tf.saved_model.save(fl_model, SAVED_MODEL_DIR, signatures=fl_model.get_signatures())
print("✅ SavedModel exported:", SAVED_MODEL_DIR)


✅ SavedModel exported: /content/drive/MyDrive/Colab_HAR_Project/converted_test_model/fl_model_saved


In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL_DIR)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]
converter.experimental_enable_resource_variables = True
tflite_model = converter.convert()

tflite_path = OUTPUT_DIR / "fl_tcn_model.tflite"
with open(tflite_path, "wb") as f:
    f.write(tflite_model)

print("✅ TFLite model exported to:", tflite_path)


✅ TFLite model exported to: /content/drive/MyDrive/Colab_HAR_Project/converted_test_model/fl_tcn_model.tflite


In [ ]:
import numpy as np
import pickle

# Create output directory
output_dir = PROJECT_DIR / "converted_test_model"
output_dir.mkdir(parents=True, exist_ok=True)

# Run predictions
X_test = X_val_scaled[:32]
y_test = y_val[:32]
preds_before = model.predict(X_test)

# Save original weights
orig_weights = [w.numpy() for w in model.trainable_variables]

# Run a federated training step
train_result = fl_model.train(X_test, y_test)

# Predict after training
preds_after = model.predict(X_test)

# Save test data and predictions (NPY)
np.save(output_dir / "X_test.npy", X_test)
np.save(output_dir / "y_test.npy", y_test)
np.save(output_dir / "preds_before.npy", preds_before)
np.save(output_dir / "preds_after.npy", preds_after)
np.save(output_dir / "orig_weights.npy", np.array(orig_weights, dtype=object), allow_pickle=True)

# Save as CSV for inspection
np.savetxt(output_dir / "X_test.csv", X_test.reshape(X_test.shape[0], -1), delimiter=",")
np.savetxt(output_dir / "y_test.csv", y_test, fmt="%d", delimiter=",")

print("✅ All outputs saved to:", output_dir)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
✅ All outputs saved to: /content/drive/MyDrive/Colab_HAR_Project/converted_test_model
